In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np
import geopandas as gpd
from shapely import wkt 

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Part I : Obtention du dataframe 
## 1.1.Récupération des loyers, des surfaces et de la localisation des résidences

In [2]:
df = pd.read_csv("fr_crous_logement_france_entiere.csv", sep =';')


In [3]:
from functions_for_data_cleaning import *

df = get_loyer(df)
df = get_surface(df)
df = get_localisation(df) 
df

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/onyxia/work/functions_for_data_cleaning.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Loyer"][j] = remove(df["Loyer"][j], 100, 1000)
/home/onyxia/work/functions_for_data_cleaning.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Surface'][j] = remove(df['Surface'][j], 7, 100)


,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Max Loyer,Min Loyer,Mean Loyer,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,537.00,333.00,435.000000,[],NaN,NaN,NaN,2.044151,48.779652,POINT (2.04415 48.77965)
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,334.00,334.00,334.000000,[],NaN,NaN,NaN,2.026887,49.048298,POINT (2.02689 49.04830)
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,359.00,332.00,345.500000,[28.0],28.0,28.0,28.000000,2.316971,48.790691,POINT (2.31697 48.79069)
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,362.00,362.00,362.000000,[18.0],18.0,18.0,18.000000,2.157896,48.698803,POINT (2.15790 48.69880)
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,154.00,154.00,154.000000,[],NaN,NaN,NaN,2.184678,48.701542,POINT (2.18468 48.70154)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,556.14,249.85,416.648333,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,POINT (-0.33921 49.20321)
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,523.00,347.00,457.333333,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,POINT (6.01534 47.23651)
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,586.00,280.00,403.571429,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,POINT (6.85371 47.63899)
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,NaN,NaN,NaN,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250)


## 1.2.Récupération des loyers privés des communes (avec la géolocalisation des communes)

In [4]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")


On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [5]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [6]:
df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)

In [7]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base). 
"""
for i in range(0,df_private.shape[0]):
    if  len(df_private["codgeo"][i]) == 4: 
            df_private["codgeo"][i] = "0" +  df_private["codgeo"][i]

In [8]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récuppère les contours géographique des communes.

In [9]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


On ne garde que le code INSEE et les données polygoniales. 

In [10]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux datafarmes

In [11]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

## 1.3. Jointures spatiales

In [12]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_3590/2580680376.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within


In [13]:
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry,index_right,LIBGEO,upr.IPm2
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,[],NaN,NaN,NaN,2.044151,48.779652,POINT (2.04415 48.77965),78297,Guyancourt,"21,73842264"
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,[],NaN,NaN,NaN,2.026887,49.048298,POINT (2.02689 49.04830),95127,Cergy,"20,65371054"
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,[28.0],28.0,28.0,28.000000,2.316971,48.790691,POINT (2.31697 48.79069),92007,Bagneux,"24,98281618"
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,[18.0],18.0,18.0,18.000000,2.157896,48.698803,POINT (2.15790 48.69880),91122,Bures-sur-Yvette,"21,2713506"
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,[],NaN,NaN,NaN,2.184678,48.701542,POINT (2.18468 48.70154),91471,Orsay,"21,07028697"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.000000,-0.339214,49.203205,POINT (-0.33921 49.20321),14327,Hérouville-Saint-Clair,"13,70540711"
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,"[19.0, 35.3]",35.3,19.0,27.150000,6.015343,47.236507,POINT (6.01534 47.23651),25056,Besançon,"13,92609502"
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-hebergement-nfc@crous-bfc.fr,03.84.21.95.37 - 06.48.41.76.36,...,"[18.0, 36.0, 51.5]",51.5,18.0,35.166667,6.853714,47.638988,POINT (6.85371 47.63899),90010,Belfort,"13,04684054"
786,416,Maret,Logements proposés : chambres traditionnelles,DIJON CENTRE,NaN,"La Résidence universitaire Maret, située en pl...",Résidence MARET \r\n 3 rue du Docteur Maret \r...,3 rue du Docteur Maret 21000 Dijon,NaN,03.45.34.84.57,...,None,NaN,NaN,NaN,5.035332,47.322498,POINT (5.03533 47.32250),21231,Dijon,"14,38634283"
